# Prueba de aleatoriedad basada en rachas totales

### Modelos no paramétricos y de regresión
##### Por: Jorge Iván Reyes Hernández

In [1]:
import numpy as np
from scipy.special import binom


In [2]:
class RunsTest:
    """ Clase para implementar la prueba de aleatoriedad basada en rachas.
    """
    def __init__(self, data):
        """ Constructor de la clase.
        
        :param data: Los mediciones a las que se les aplicará la prueba de aleatoriedad.
        """
        self.data = data
        self._assert_binary_data()
        self.type_i = None
        self.type_ii = None
        self.n_1: int = 0
        self.n_2: int = 0
        self._get_types()
        self.n: int = self.n_1 + self.n_2
        self.r: int = 1
        self._get_runs()
        self.significance_level = 0.0 
    
    def _get_types(self):
        """
        Obtenemos los dos tipos de datos.
        Se elige que los objetos del tipo 1 sea de los que menos hay.
        """
        self.type_i, self.type_ii = set(self.data)
        
        for i in self.data:
            if i == self.type_i:
                self.n_1 += 1
            else:
                self.n_2 += 1
        if self.n_1 > self.n_2:
            self.n_1, self.n_2 = self.n_2, self.n_1
            self.type_i, self.type_ii = self.type_ii, self.type_i
        
    def _assert_binary_data(self):
        """
        Verifica que los datos sean dicotómicos.
        """
        assert len(set(self.data)) == 2
        
    def _get_runs(self):
        """
        Calcula la cantidad de rachas observadas.
        """
        cur = self.data[0]
        for i in range(1, self.data.size):
            if cur != self.data[i]:
                self.r += 1
            cur = self.data[i]
            
    def process_non_binary_data(self):
        """
        Implementa la opción para pasarle datos no binarios eligiendo un punto de corte para
        volverlos binarios.
        """
        pass
    
    def _test_statistics_asymptotic(self):
        """
        Implementa la distribución asintótica de la estadística de prueba.
        """
        pass
    
    def _pmf_even(self, r):
        """
        Función de probabilidad de la estadística de prueba cuando r es par.
        
        :param r: Valor que toma R.
        :return: La probabilidad de que R=r
        """
        num = 2 * binom(self.n_1 - 1, r / 2 - 1) * binom(self.n_2 - 1, r / 2 - 1)
        den = binom(self.n, self.n_1)
        return num / den
    
    def _pmf_odd(self, r):
        """
        Función de probabilidad de la estadística de prueba cuando r es impar.
        
        :param r: Valor que toma R.
        :return: La probabilidad de que R=r
        """
        num = binom(self.n_1 - 1, (r - 1) / 2) * binom(self.n_2 - 1, (r - 3) / 2)
        num += binom(self.n_1 - 1, (r - 3) / 2) * binom(self.n_2 - 1, (r - 1) / 2)
        den = binom(self.n, self.n_1)
        return num / den
        
    def _test_statistic_exact_two_sized(self, alpha):
        """
        Calcula las constantes (cuántiles) de la pmf de R tales que la proba acumulada
        sea menor que el nivel de significancia para la prueba de dos colas.
        
        :param alpha: Nivel de significancia deseado.
        :return: Las constantes que definen la región crítica.
        """
        prob = 0.0
        k_1 = 1
        k_2 = self.n + 1
        while prob <= alpha and (k_1 <= k_2):
            self.significance_level = prob
            k_1 += 1
            k_2 -= 1
            p_1 = self._pmf_even(k_1) if (k_1 % 2 == 0) else self._pmf_odd(k_1)
            p_2 = self._pmf_even(k_2) if (k_2 % 2 == 0) else self._pmf_odd(k_2)
            prob += p_1 + p_2
        k_1 -= 1
        k_2 += 1
        return k_1, k_2
        
    def run_test(self, alpha=0.05, alternative="two-sized", exact=True, cutoff=None):
        if exact and alternative == "two-sized":
            k_1, k_2 = self._test_statistic_exact_two_sized(alpha)
            print(f"Rechazar H_0 si: R <= {k_1} o si R => {k_2}")
            print(f"Valor que tomó la estadística: R_obs = {self.r}")
            decision = "-> Rechazamos H_0" if (self.r <= k_1 or self.r >= k_2) else "-> No rechazamos H_0"
            print(decision + f" con nivel de significancia {self.significance_level:.3f}")
        elif not exact and alternative == "two-sized":  
            pass
        else:
            pass
    

###### Ejemplo de las notas a mano

In [3]:
data = np.array([63, 67, 66, 63, 61, 44, 60, 69, 71, 62, 67, 55, 71, 55, 65, 65, 61, 60, 67])
sample_mean = data.mean()
binary_data = []

for d in data:
    if d < sample_mean:
        binary_data.append("-")
    elif d > sample_mean:
        binary_data.append("+")
        
binary_data = np.array(binary_data)

print(sample_mean)

62.73684210526316


In [4]:
binary_data


array(['+', '+', '+', '+', '-', '-', '-', '+', '+', '-', '+', '-', '+',
       '-', '+', '+', '-', '-', '+'], dtype='<U1')

In [5]:
tester = RunsTest(data=binary_data)
tester.run_test()


Rechazar H_0 si: R <= 5 o si R => 16
Valor que tomó la estadística: R_obs = 11
-> No rechazamos H_0 con nivel de significancia 0.013


###### Ejemplo ayudantía

In [6]:
data = np.array(["U", "U", "U", "U", "U", "L", "L", "L", "L", "L", "L", "U", "L", "U", "L", "U"])

In [7]:
tester = RunsTest(data=data)

In [8]:
tester.data

array(['U', 'U', 'U', 'U', 'U', 'L', 'L', 'L', 'L', 'L', 'L', 'U', 'L',
       'U', 'L', 'U'], dtype='<U1')

In [9]:
tester.run_test()

Rechazar H_0 si: R <= 4 o si R => 14
Valor que tomó la estadística: R_obs = 7
-> No rechazamos H_0 con nivel de significancia 0.018


#### Ejercicio 1 (ayudantías)


In [10]:
data = np.array(["A", "A", "A", "B", "B", "A", "A", "A", "A", "B", "A", "A", "B", "A", "A", "A", "A", "B", "B", "B", "B"])

In [11]:
data

array(['A', 'A', 'A', 'B', 'B', 'A', 'A', 'A', 'A', 'B', 'A', 'A', 'B',
       'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B'], dtype='<U1')

In [12]:
tester = RunsTest(data=data)
tester.run_test()

Rechazar H_0 si: R <= 6 o si R => 17
Valor que tomó la estadística: R_obs = 8
-> No rechazamos H_0 con nivel de significancia 0.020


## Usando statsmodels

In [13]:
from statsmodels.sandbox.stats.runs import runstest_1samp 

In [14]:
data = np.array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1])

In [15]:
z, p_val = runstest_1samp(data, correction=False)

In [16]:
p_val

0.3006229881969068

Si $\alpha = 0.05$, entonces $p_{val} = 0.3 \leq 0.05 = \alpha$ no se cumple, por lo que no rechazamos la hipótesis nula.

## Ejercicio
 
Los datos "datos_historicos.csv" fueron extraídos de https://mx.investing.com/currencies/usd-mxn-historical-data. ¿Son estos datos aleatorios o presentan alguna tendencia?